In [2]:
import numpy as np
import argparse
import pickle
import cv2
import os
import time
from keras.models import load_model
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import requests
from datetime import datetime
import pytz
import folium
from geopy.geocoders import Nominatim
from selenium import webdriver

SGT = pytz.timezone('Asia/Singapore')
singapore_tz = datetime.now(SGT)
curr_date = singapore_tz.strftime("%d-%m-%Y")
curr_time = (datetime.now(SGT).strftime("%H:%M:%S"))

telegram_auth_token = "6336511764:AAG3aM1bJ6wXQ_7EG8ZgpLucpmZmGCxINX8"
telegram_group_chat_id = "-1001926862448"

if not os.path.exists('output'):
    os.mkdir('output')

print("Loading model ...")
model = load_model('/Users/faizahkureshi/Desktop/GAIP Project/modelnew.h5')
Q = deque(maxlen=128)
vs = cv2.VideoCapture(0)


writer = None
(W, H) = (None, None)
count = 0
counter = 0

def send_message_to_group(message):
    telegram_api_url = f"https://api.telegram.org/bot{telegram_auth_token}/sendMessage"
    params = {
        "chat_id": telegram_group_chat_id,
        "text": message
    }
    response = requests.get(telegram_api_url, params=params)
    if response.status_code == 200:
        print("Message sent successfully!")
    else:
        print("Failed to send the message.")

def send_image(message, photo_path=None):
    telegram_api_url = f"https://api.telegram.org/bot{telegram_auth_token}/sendPhoto"

    params = {
        "chat_id": telegram_group_chat_id,
        "caption": message
    }

    files = None
    if photo_path:
        files = {"photo": open(photo_path, "rb")}

    response = requests.post(telegram_api_url, params=params, files=files)
    if response.status_code == 200:
        print("Image sent successfully!")
    else:
        print("Failed to send the image.")

location = "SIngapore"

def find_loc():
    # Create a geocoder object
    geolocator = Nominatim(user_agent="my-app")

    # Specify the location you want to geocode

    # Geocode the location
    geocode_result = geolocator.geocode(location)

    # Print the place name, latitude, and longitude
    lat=geocode_result.latitude
    long=geocode_result.longitude
    loc(lat,long)


def loc(lat,long):
    map = folium.Map(location=[lat, long], zoom_start=15)
    folium.Marker(location=[lat, long], popup="Location").add_to(map)
    map.save("map.html")
#from google.colab.patches import cv2_imshow
#fig=plt.figure(figsize=(16, 30))


from geopy.geocoders import Nominatim

def get_latitude_longitude(location):
    # Create a geocoder object
    geolocator = Nominatim(user_agent="my-app")

    # Geocode the location
    geocode_result = geolocator.geocode(location)

    # Check if geocoding was successful
    if geocode_result is None:
        print(f"Failed to geocode location: {location}")
        return None

    # Extract latitude and longitude
    latitude = geocode_result.latitude
    longitude = geocode_result.longitude

    return latitude, longitude

latitude, longitude = get_latitude_longitude(location)

driver = webdriver.Chrome()  # Replace with the appropriate WebDriver for your browser

find_loc()
html_file_path = '/Users/faizahkureshi/Desktop/GAIP Project/map.html'
driver.get(f'file://{html_file_path}')

# Take a screenshot of the rendered HTML page
screenshot_path = '/Users/faizahkureshi/Desktop/GAIP Project/output/screenshot.png'
driver.save_screenshot(screenshot_path)
# Close the browser and WebDriver
driver.quit()

while True:
    ret, frame = vs.read()

    if not ret:
        break

    if W is None or H is None:
        (H, W) = frame.shape[:2]

    output = frame.copy()

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (128, 128)).astype("float32")
    frame = frame.reshape(128, 128, 3) / 255

    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)

    results = np.array(Q).mean(axis=0)
    i = (preds > 0.50)[0]
    label = i
    print(i)
    text_color = (0, 255, 0)  # default: green

    if label:  # Violence prob
        text_color = (0, 0, 255)  # red

    if label:
        print("1")
        text = "Violence: {}".format(label)
        FONT = cv2.FONT_HERSHEY_SIMPLEX

        cv2.putText(output, text, (35, 50), FONT, 1.25, text_color, 3)

        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter("output/v_output.avi", fourcc, 30, (W, H), True)

        writer.write(output)

        image_path = f"output/output_{counter}.png"
        plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
        plt.savefig(image_path)
        plt.close()

        message = f"Alert! Violence detected on {curr_date} at {curr_time}  Location: Latitude- {latitude}  Longitude- {longitude}."
        send_message_to_group(message)

        send_image("Check out this image!", photo_path=image_path)
        send_image("Check out this map!", photo_path=screenshot_path)


        counter += 1

    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

print("[INFO] Cleaning up...")
writer.release()
vs.release()


Loading model ...


KeyboardInterrupt: 